In [ ]:
%reload_ext autoreload
%autoreload 2
%cd ..

In [ ]:
HOPSWORKS_PROJECT_NAME = 'ny_taxi_demand_predictor'

In [ ]:
import os
from dotenv import load_dotenv
from src.paths import PARENT_DIR

In [ ]:
load_dotenv(PARENT_DIR / '.env')
HOPSWORKS_API_KEY = os.environ['HOPSWORKS_API_KEY']

In [ ]:
from datetime import datetime
import pandas as pd
from src.data import load_raw_data

In [ ]:
from_year = 2022
to_year = datetime.now().year

In [ ]:
print(f'Downloading raw data from {from_year} to {to_year}')

rides = pd.DataFrame()
for year in range(from_year, to_year + 1):

    # download data for the whole year
    rides_one_year = load_raw_data(year)

    # append data to the total rides dataframe
    rides = pd.concat([rides, rides_one_year])

In [ ]:
print(f'{len(rides)=:,}')

In [ ]:
from src.data import transform_raw_data_into_ts_data

ts_data = transform_raw_data_into_ts_data(rides)

In [ ]:
# string to datetime
ts_data['pickup_hour'] = pd.to_datetime(ts_data['pickup_hour'], utc=True)

# add column with Unix epoch milliseconds
ts_data['pickup_ts'] = ts_data['pickup_hour'].astype(int) // 10**6

In [ ]:
import hopsworks

In [ ]:
project = hopsworks.login(
    project=HOPSWORKS_PROJECT_NAME,
    api_key_value=HOPSWORKS_API_KEY
)

In [ ]:
feature_store = project.get_feature_store()

In [ ]:
FEATURE_GROUP_NAME = 'time_series_hourly_feature_group'
FEATURE_GROUP_VERSION = 1

In [ ]:
feature_group = feature_store.get_or_create_feature_group(
    name=FEATURE_GROUP_NAME,
    version=FEATURE_GROUP_VERSION,
    description='Time series of NYC taxi rides at hourly intervals',
    primary_key=['pickup_location_id', 'pickup_hour'],
    event_time='pickup_hour',
)

In [ ]:
feature_group.insert(ts_data, write_options={'wait_for_job': False})